In [375]:
import glob
import os
import numpy as np
import pickle
import math as mt
import shutil

In [376]:
def create_windows_antennas(csi_list, labels_list, labels_PI_list, sample_length, stride_length, remove_mean=False):
    csi_matrix_stride = []
    labels_stride = []
    labels_PI_stride = []
    for i in range(len(labels_list)):
        csi_i = csi_list[i]
        label_i = labels_list[i]
        label_PI_i = labels_PI_list[i]
        len_csi = csi_i.shape[2]
        for ii in range(0, len_csi - sample_length, stride_length):
            csi_wind = csi_i[:, :, ii:ii + sample_length, ...]
            if remove_mean:
                csi_mean = np.mean(csi_wind, axis=2, keepdims=True)
                csi_wind = csi_wind - csi_mean
            csi_matrix_stride.append(csi_wind)
            labels_stride.append(label_i)
            labels_PI_stride.append(label_PI_i)
    return csi_matrix_stride, labels_stride, labels_PI_stride

In [377]:
def convert_to_number(lab, csi_label_dict):
    lab_num = np.argwhere(np.asarray(csi_label_dict) == lab)[0][0]
    return lab_num

def convert_PI_to_number(lab):
    """
    Args: 
        lab (str): label of the person. Can be "1","2",...,"7".
    Return: 
        int: corresponding label
    """
    label_mapping = {
        "1": 1,
        "2": 1,
        "4": 1,
        "6": 1,
        "3": 2,
        "5": 2,
        "7": 3
    }
    
    if lab not in label_mapping:
        raise ValueError(f"Label '{lab}' is not valid. Valid labels are {list(label_mapping.keys())}.")
    
    return label_mapping[lab]

In [378]:
convert_PI_to_number("3")

2

In [379]:
dir_init = './doppler_traces/' #Directory of data
subdir_init = 'S1a,S1b,S1c,S2a,S2b,S3a,S4a,S4b,S5a,S6a,S6b,S7a' #Sub-directories
sample_lengths_init = 31 #Number of packets in a sample
sliding_init = 1 #Number of packet for sliding operations
windows_length_init = 340 #Number of samples per window
stride_lengths_init = 30 #Number of samples to stride
labels_activities_init = 'E,L,W,R,J' #Labels of the activities to be considered
# wtf does this mean?
n_tot_init = 4

In [380]:
labels_activities = labels_activities_init
csi_label_dict = []
for lab_act in labels_activities.split(','):
    csi_label_dict.append(lab_act)

activities = np.asarray(labels_activities)

In [381]:
csi_label_dict

['E', 'L', 'W', 'R', 'J']

In [382]:
n_tot = n_tot_init
num_packets = sample_lengths_init  # 51
middle = int(np.floor(num_packets / 2))
list_subdir = subdir_init  # string

In [383]:
for subdir in list_subdir.split(','):
    exp_dir = dir_init + subdir + '/'

    path_train = exp_dir + 'train_antennas_' + str(activities)
    path_val = exp_dir + 'val_antennas_' + str(activities)
    path_test = exp_dir + 'test_antennas_' + str(activities)
    paths = [path_train, path_val, path_test]
    for pat in paths:
        if os.path.exists(pat):
            remove_files = glob.glob(pat + '/*')
            for f in remove_files:
                os.remove(f)
        else:
            os.mkdir(pat)

    path_complete = exp_dir + 'complete_antennas_' + str(activities)
    if os.path.exists(path_complete):
        shutil.rmtree(path_complete)

    names = []
    all_files = os.listdir(exp_dir)
    for i in range(len(all_files)):
        if all_files[i].startswith('S'):
            names.append(all_files[i][:-4])
    names.sort()

    # add label for Person Identification
    csi_matrices = []
    labels = []
    labels_PI = []
    lengths = []
    label = 'null'
    label_PI = 'null'
    prev_label = label
    prev_label_PI = label_PI
    csi_matrix = []
    processed = False
    for i_name, name in enumerate(names):
        if i_name % n_tot == 0 and i_name != 0 and processed:
            ll = csi_matrix[0].shape[1]

            for i_ant in range(1, n_tot):
                if ll != csi_matrix[i_ant].shape[1]:
                    break
            lengths.append(ll)
            csi_matrices.append(np.asarray(csi_matrix))
            labels.append(label)
            labels_PI.append(label_PI)
            csi_matrix = []

        label = name[4]
        label_PI = name[1]
        print("name:", name, "label:", label, "label_PI:", label_PI)

        if label not in csi_label_dict:
            processed = False
            continue
        processed = True

        print(name)

        label = convert_to_number(label, csi_label_dict)
        label_PI = convert_PI_to_number(label_PI)
        if i_name % n_tot == 0:
            prev_label = label
            prev_label_PI = label_PI
        elif label != prev_label:# or label_PI != prev_label_PI:
            print('error in ' + str(name))
            break

        name_file = exp_dir + name + '.txt'
        with open(name_file, "rb") as fp:  # Unpickling
            stft_sum_1 = pickle.load(fp)

        stft_sum_1_mean = stft_sum_1 - np.mean(stft_sum_1, axis=0, keepdims=True)

        csi_matrix.append(stft_sum_1_mean.T)
    
    print("name:", name, "label:", label, "label_PI:", label_PI)
    error = False
    if processed:
        # for the last block
        if len(csi_matrix) < n_tot:
            print('error in ' + str(name))
        ll = csi_matrix[0].shape[1]

        for i_ant in range(1, n_tot):
            if ll != csi_matrix[i_ant].shape[1]:
                print('error in ' + str(name))
                error = True
        if not error:
            lengths.append(ll)
            csi_matrices.append(np.asarray(csi_matrix))
            labels.append(label)
            labels_PI.append(label_PI)

    print("labels:", labels)
    print("labels_PI:", labels_PI)
    
    if not error:
        lengths = np.asarray(lengths)
        length_min = np.min(lengths)

        csi_train = []
        csi_val = []
        csi_test = []
        length_train = []
        length_val = []
        length_test = []
        for i in range(len(labels)):
            ll = lengths[i]
            train_len = int(np.floor(ll * 0.6))
            length_train.append(train_len)
            csi_train.append(csi_matrices[i][:, :, :train_len])

            start_val = train_len + mt.ceil(num_packets/sliding_init)
            val_len = int(np.floor(ll * 0.2))
            length_val.append(val_len)
            csi_val.append(csi_matrices[i][:, :, start_val:start_val + val_len])

            start_test = start_val + val_len + mt.ceil(num_packets/sliding_init)
            length_test.append(ll - val_len - train_len - 2*mt.ceil(num_packets/sliding_init))
            csi_test.append(csi_matrices[i][:, :, start_test:])

        window_length = windows_length_init  # number of windows considered
        stride_length = stride_lengths_init

        list_sets_name = ['train', 'val', 'test']
        list_sets = [csi_train, csi_val, csi_test]
        list_sets_lengths = [length_train, length_val, length_test]

        for set_idx in range(3):
            csi_matrices_set, labels_set, labels_set_PI = create_windows_antennas(list_sets[set_idx], labels, labels_PI, window_length,
                                                                    stride_length, remove_mean=False)

            num_windows = np.floor((np.asarray(list_sets_lengths[set_idx]) - window_length) / stride_length + 1)
            if not len(csi_matrices_set) == np.sum(num_windows):
                print('ERROR - shapes mismatch')
                print(len(csi_matrices_set), ' and ', np.sum(num_windows), '\n')

            names_set = []
            suffix = '.txt'
            for ii in range(len(csi_matrices_set)):
                name_file = exp_dir + list_sets_name[set_idx] + '_antennas_' + str(activities) + '/' + \
                            str(ii) + suffix
                names_set.append(name_file)
                with open(name_file, "wb") as fp:  # Pickling
                    pickle.dump(csi_matrices_set[ii], fp)

            name_labels = exp_dir + '/labels_' + list_sets_name[set_idx] + '_antennas_' + str(activities) + suffix
            with open(name_labels, "wb") as fp:  # Pickling
                pickle.dump(labels_set, fp)

            name_labels_PI = exp_dir + '/labels_PI_' + list_sets_name[set_idx] + '_antennas_' + str(activities) + suffix
            with open(name_labels_PI, "wb") as fp:  # Pickling
                pickle.dump(labels_set_PI, fp)

            name_f = exp_dir + '/files_' + list_sets_name[set_idx] + '_antennas_' + str(activities) + suffix
            with open(name_f, "wb") as fp:  # Pickling
                pickle.dump(names_set, fp)

            name_f = exp_dir + '/num_windows_' + list_sets_name[set_idx] + '_antennas_' + str(activities) + suffix
            with open(name_f, "wb") as fp:  # Pickling
                pickle.dump(num_windows, fp)

name: S1a_C_stream_0 label: C label_PI: 1
name: S1a_C_stream_1 label: C label_PI: 1
name: S1a_C_stream_2 label: C label_PI: 1
name: S1a_C_stream_3 label: C label_PI: 1
name: S1a_E_stream_0 label: E label_PI: 1
S1a_E_stream_0
name: S1a_E_stream_1 label: E label_PI: 1
S1a_E_stream_1
name: S1a_E_stream_2 label: E label_PI: 1
S1a_E_stream_2
name: S1a_E_stream_3 label: E label_PI: 1
S1a_E_stream_3
name: S1a_H_stream_0 label: H label_PI: 1
name: S1a_H_stream_1 label: H label_PI: 1
name: S1a_H_stream_2 label: H label_PI: 1
name: S1a_H_stream_3 label: H label_PI: 1
name: S1a_J1_stream_0 label: J label_PI: 1
S1a_J1_stream_0
name: S1a_J1_stream_1 label: J label_PI: 1
S1a_J1_stream_1
name: S1a_J1_stream_2 label: J label_PI: 1
S1a_J1_stream_2
name: S1a_J1_stream_3 label: J label_PI: 1
S1a_J1_stream_3
name: S1a_J2_stream_0 label: J label_PI: 1
S1a_J2_stream_0
name: S1a_J2_stream_1 label: J label_PI: 1
S1a_J2_stream_1
name: S1a_J2_stream_2 label: J label_PI: 1
S1a_J2_stream_2
name: S1a_J2_stream_3 l

The number of labels created is equal to the number of .txt files generated, thus the labels are not expanded.

In [389]:
# Define the file name
S1a_name_f = '/home/giovanni/Desktop/Deep Learning/project/NNDL HAR project/doppler_traces/S1a/labels_train_antennas_E,L,W,R,J.txt'

name_f = '/home/giovanni/Desktop/Deep Learning/project/NNDL HAR project/doppler_traces/S7a/labels_train_antennas_E,L,W,R,J.txt'

PI_name_f = '/home/giovanni/Desktop/Deep Learning/project/NNDL HAR project/doppler_traces/S7a/labels_PI_train_antennas_E,L,W,R,J.txt'

with open(name_f, "rb") as fp:
    labels_set = pickle.load(fp)

with open(PI_name_f, "rb") as fp:
    labels_PI_set = pickle.load(fp)

print("number of training labels:", len(labels_set))
print("number of training labels:", len(labels_PI_set))

number of training labels: 1689
number of training labels: 1689


In [390]:
print(labels_PI_set)

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 